# This is Estelle's notebook no.2

In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
import cv2
# import pytesseract

import requests
from bs4 import BeautifulSoup
import httpx

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

from sklearn.utils import shuffle

In [3]:
from PIL import Image
from io import BytesIO

In [4]:
from pokedex.utils_dataset import reduce_dataset
from pokedex.augmentation import get_augment_data, transform_array, generate_augmented_image, apply_blur

2024-05-22 10:13:08.983018: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Reduce dataset function creation

In [60]:
setinfo = np.array(
        [['dv1', '21', 'Dragon Vault', 'right'],
         ['swsh9', '186', 'Brilliant Stars', 'left'],
         ['swsh45', '73', 'Shining Fates', 'left'],
         ['swsh6', '233', 'Chilling Reign', 'left'],
         ['swsh12pt5', '160', 'Crown Zenith', 'left'],
         ['xy1', '146', 'XY', 'right'],
         ['xy2', '110', 'Flashfire', 'right'],
         ['xy3', '114', 'Furious Fists', 'right'],
         ['g1', '117', 'Generations', 'right'],
         ['xy4', '124', 'Phantom Forces', 'right'],
         ['xy6', '112', 'Roaring Skies', 'right'],
         ['xy7', '100', 'Ancient Origins', 'right'],
         ['dp1', '130', 'Diamond & Pearl', 'right'],
         ['dp2', '124', 'Mysterious Treasures', 'right'],
         ['sm4', '126', 'Crimson Invasion', 'left'],
         ['swsh10', '216', 'Astral Radiance', 'left'],
         ['sv4', '266', 'Paradox Rift', 'left'],
         ['sv3pt5', '207', '151', 'left'],
         ['sv3', '230', 'Obsidian Flames', 'left'],
         ['sv2', '279', 'Paldea Evolved', 'left']])

In [61]:
df = pd.read_json('../raw_data/dict_dataset_full.json')
df['corner']=[np.array(v) for v in df['corner']]

In [195]:
setinfo_left = setinfo[setinfo[:,3] == 'left']
setinfo_right = setinfo[setinfo[:,3] == 'right']

In [283]:
set_list = setinfo_right[:,0]
set_list = np.append(set_list, 'no') 
# set_list[-1] = 'no'
set_list

array(['dv1', 'xy1', 'xy2', 'xy3', 'g1', 'xy4', 'xy6', 'xy7', 'dp1',
       'dp2', 'no'], dtype='<U20')

In [306]:
nb_cards = np.array([sum((df['set_id'] == set_list[i]) & (df['position'] == 'right')) for i in range(set_list.shape[0])])

set_to_dropcards = np.vstack((set_list[nb_cards > 150], nb_cards[nb_cards > 150]))
set_to_dropcards

array([['no'],
       ['1974']], dtype='<U21')

In [311]:
df_small = pd.DataFrame()

for i in range(set_to_dropcards.shape[1]):
    idx = np.random.randint(0, high=int(set_to_dropcards[1,i]), size=150, dtype=int)
    # df_small = df[df['set_id'] == set_to_dropcards[0,i]].iloc[idx]
    df_small = pd.concat([ df_small, df[(df['set_id'] == set_to_dropcards[0,i]) & (df['position'] == 'right')].iloc[idx] ], axis=0, ignore_index=True)


In [318]:
for i in range(len(set_list[nb_cards <= 150])):
    df_small = pd.concat([ df_small, df[df['set_id'] == set_list[nb_cards <= 150][i]] ], axis=0, ignore_index=True)

In [317]:
df_small

,corner,position,set_id,set_name
0,"[[145, 145, 145, 145, 145, 145, 145, 145, 145,...",right,no,no
1,"[[154, 154, 154, 154, 154, 154, 154, 154, 154,...",right,no,no
2,"[[191, 191, 191, 191, 191, 191, 191, 191, 191,...",right,no,no
3,"[[154, 154, 154, 154, 154, 154, 154, 154, 154,...",right,no,no
4,"[[154, 154, 154, 154, 154, 154, 154, 154, 154,...",right,no,no
...,...,...,...,...
1203,"[[198, 198, 198, 199, 199, 200, 200, 200, 202,...",right,dp2,Mysterious Treasures
1204,"[[156, 155, 156, 157, 153, 145, 138, 135, 127,...",right,dp2,Mysterious Treasures
1205,"[[130, 129, 126, 121, 115, 110, 109, 109, 105,...",right,dp2,Mysterious Treasures
1206,"[[133, 121, 126, 120, 119, 121, 107, 110, 91, ...",right,dp2,Mysterious Treasures


In [62]:
reduce_dataset()

## Work with left and right dataset

In [34]:
# df_right = pd.read_json('../raw_data/dict_reduceddataset_right.json')
# df_left = pd.read_json('../raw_data/dict_reduceddataset_left.json')

In [5]:
df_right = get_augment_data('../raw_data/dict_reduceddataset_right.json')

In [8]:
df_right

,corner,position,set_id,set_name
0,"[[[[154], [154], [154], [154], [154], [154], [...",right,no,no
1,"[[[[154], [154], [154], [154], [154], [154], [...",right,no,no
2,"[[[[191], [191], [191], [191], [191], [191], [...",right,no,no
3,"[[[[154], [154], [154], [154], [154], [154], [...",right,no,no
4,"[[[[153], [153], [153], [154], [154], [154], [...",right,no,no
...,...,...,...,...
3295,"[[[190], [190], [190], [190], [190], [190], [1...",right,dv1,Dragon Vault
3296,"[[[190], [190], [190], [190], [190], [190], [1...",right,dv1,Dragon Vault
3297,"[[[190], [190], [190], [190], [190], [190], [1...",right,dv1,Dragon Vault
3298,"[[[190], [190], [190], [190], [190], [190], [1...",right,dv1,Dragon Vault


### Split data into train and validation

In [9]:
df_right.shape

(3300, 4)

In [16]:
df_right = shuffle(df_right).reset_index(drop=True)

In [17]:
df_right

,corner,position,set_id,set_name
0,"[[[[190], [190], [190], [190], [190], [190], [...",right,xy1,XY
1,"[[[225], [226], [229], [231], [232], [233], [2...",right,g1,Generations
2,"[[[190], [190], [190], [190], [190], [190], [1...",right,xy6,Roaring Skies
3,"[[[190], [190], [190], [190], [190], [190], [1...",right,xy3,Furious Fists
4,"[[[154], [154], [154], [154], [154], [154], [1...",right,no,no
...,...,...,...,...
3295,"[[[[190], [190], [190], [190], [190], [190], [...",right,xy2,Flashfire
3296,"[[[190], [190], [190], [190], [190], [190], [1...",right,xy2,Flashfire
3297,"[[[190], [190], [190], [190], [190], [190], [1...",right,xy2,Flashfire
3298,"[[[190], [190], [190], [190], [190], [190], [1...",right,xy6,Roaring Skies


In [18]:
nb_split = int(df_right.shape[0]*0.8)
nb_split

2640

In [19]:
dfr_train = df_right.iloc[:nb_split]
dfr_val = df_right.iloc[nb_split:]

In [28]:
y_train = dfr_train.loc[:,'set_id']
y_train.shape
y_val = dfr_val.loc[:,'set_id']
y_val.shape

(660,)

In [38]:
X_train = dfr_train.loc[:,'corner']
X_train[0].shape

(1, 72, 184, 1)

In [48]:
XX_train = np.empty((nb_split, 72, 184))

for i, x in enumerate(X_train):
    XX_train[i,:,:,:] = x

XX_train

(1, 72, 184, 1)
[[[[0.74509804]
   [0.74509804]
   [0.74509804]
   ...
   [0.65098039]
   [0.65098039]
   [0.6745098 ]]

  [[0.74509804]
   [0.74509804]
   [0.74509804]
   ...
   [0.65098039]
   [0.65098039]
   [0.6745098 ]]

  [[0.74509804]
   [0.74509804]
   [0.74509804]
   ...
   [0.65882353]
   [0.65098039]
   [0.66666667]]

  ...

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.74509804]
   [0.74509804]
   [0.74509804]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.74509804]
   [0.74509804]
   [0.74509804]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.74509804]
   [0.74509804]
   [0.74509804]]]]
(72, 184, 1)
[[[0.88235294]
  [0.88627451]
  [0.89803922]
  ...
  [0.59607843]
  [0.67843137]
  [0.70980392]]

 [[0.89019608]
  [0.89411765]
  [0.90196078]
  ...
  [0.59215686]
  [0.6745098 ]
  [0.70588235]]

 [[0.89803922]
  [0.90196078]
  [0.90980392]
  ...
  [0.58431373]
  [0.66666667]
  [0.69803922]]

 ...

 [[0.        ]
  [0.        ]
  [

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

## Model baseline

In [55]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

In [59]:
# ——— CNN Version III:

# - Integrate Preprocessing Layers
# - Add Regularization

model_pipe = Sequential([
    layers.Reshape((72, 184, 1), input_shape=(72,184)),
    layers.Rescaling(scale=1./255.),
    
    layers.Conv2D(32, (5,5), padding='same', activation="relu"),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(32, (3,3), padding='same', activation="relu"),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(11, activation='softmax')
])


In [ ]:
model_pipe.compile(loss='sparse_categorical_crossentropy', # No need to OHE target
                   optimizer='adam',
                   metrics=['accuracy'])
model_pipe.fit(X_train, y_train,
               epochs=1, # Use early stopping in practice
               batch_size=32,
               verbose=1)